<a href="https://colab.research.google.com/github/darkway30/darkway30-Retail_Forecasting/blob/master/UKOnline_transactions%20forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

Part one: We start Understaning our Data.

In [ ]:
# Readin our CSV file trasactions to start analysing the prameters
data = pd.read_csv('Online_Retail.csv', encoding = 'unicode_escape')


In [ ]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [ ]:
# Whats the shape of our data
# This Dataset contains 532619 rows and 8 columns
data.shape

(541909, 8)

In [ ]:
# Data information
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


From the data information insight InvoiceNo should be int64; there must be something incorrect with this variable. When attempting to use df.InvoiceNo.astype('int64') I receive an error. it's impossible to convert str into an int, meaning wrong listings in the data. 

We can recognize that there are 532619 entries from Data Information, and these are the transactions I'll try to analyze. 
There is two float type column ('UnitPrice', 'CustomerID)
An integer type column ('Quantity'). 
Five object type column ('InvoiceNo',StockCode','Description','InvoiceDate','Country') 

In [ ]:
# Description to look for any Null Values before cleaning
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [ ]:
Null_percentage = data.isnull().sum() / data.shape[0] * 100
Null_percentage

InvoiceNo       0.000000
StockCode       0.000000
Description     0.268311
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     24.926694
Country         0.000000
dtype: float64

Approximately 25 % of the customers are unknown or Null, and 0.2% items were missing; these values that we'll have to remove to better understand the actual data. 

In [ ]:
# now we can get an idea about the actual numbers
data.isnull().sum().sort_values(ascending=False)

CustomerID     135080
Description      1454
Country             0
UnitPrice           0
InvoiceDate         0
Quantity            0
StockCode           0
InvoiceNo           0
dtype: int64

With this information, we need to point to the business owner that we're missing a lot 25% is an immense value to miss. Something they can work on recovering.

We move to the next step: Cleaning our data to be more comfortable and more precise to analyze.

In [ ]:
# Remving the missing values with Panda Drop format
data = data.dropna()
data.shape

(406829, 8)

In [ ]:
# making sure that our data is not missing in more values
data.isnull().sum().sum()

0

Part three: let's explore our parameters
Countries/Stock that we can Visualize later to understand the countries we should focus on and the Items sold.

Invoices**bold text**

In [ ]:
# How many Unique Invoices made in our 40+ Data
data.InvoiceNo.nunique()

22190

Stock items

In [ ]:
# How many Unique Invoices made in our 40+ Data
data.StockCode.nunique()

3684

Descriptions

In [ ]:
data.Description.nunique()

3896

Since we have only 22190 Unique invoices, let's analyze our customers and look at their total number of transactions made this year and their last purchase.
However, in a later step, a NaN Description shows us a failed transaction, so we will have to deal with NaN values to analyze our customers better.

In [ ]:
# Changing our CustomerID data to get a more accurate results.

In [ ]:
data['CustomerID'] = data['CustomerID'].astype(str)

In [ ]:
# amount = [the items sold] x [price]
data['Amount'] = data['Quantity']*data['UnitPrice']
# pandas. DataFrame. groupby to sum our to values
total_t = data.groupby('CustomerID')['Amount'].sum()
total_t = total_t.reset_index()

In [ ]:
# let's explore our first attribute
total_t.head()


,CustomerID,Amount
0,12346.0,0.00
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40
